# Writing a spelling corrector

By Saurabh Mathur

Spelling correctors are found nearly everywhere - Search engines, smartphone keyboards, word processors and so on. 

_But how do they work ? _

![](images/did-you-mean.png)

**Peter Norvig**, the Director of Research at Google, wrote a blog post explaining how.

This is an abridged version of the same. The article can be found here - [http://norvig.com/spell-correct.html](http://norvig.com/spell-correct.html)

![](images/peter-norvig.jpg)

## A bit of math

Formally, our problem can be stated as follows, 

>_Given a word, choose the most likely spelling correction for that word _


To accomplish that we need to find the correction (`c`) with the maximum value of the probability given the word (`w`). That is, 

```
max P(c|w)
```

Applying Bayes' Theorem, we can expand `P(c|w)` as

```
P(c|w) = P(w|c) P(c) / P (w)
```

## Counting words

First of all, we need to find the probability of the occurance of a correct word. That is,
```
P(c)
```

In [68]:
import os, re, pprint

word_counts = {} # ex. { 'kingdom': 5, 'province': 10, ..  }
filepath = os.path.join('data', 'big.txt') # 'data/big.txt' (linux/mac) or 'data\big.txt' (windows)

with open(filepath) as f:
    alltext = f.read() # read entire file as string
    alltext_lower = alltext.lower() # file contents as lowercase
    
    # '"seven!" i answered.' => ['seven', 'i', 'answered']
    words = re.findall('[a-z]+', alltext_lower) 
    
    for word in words:
        if word in word_counts:
            word_counts[word] += 1
        else:
            word_counts[word] = 1
            
print ('{} words read.'.format(len(word_counts)))

29157 words read.


## Finding the correct word

Next, we need to find how to correct a particular word. To accomplish this, we will find the all "words" that are candidate corrections.

For simplicity, we will to find all the words that can be formed from our misspelt word by changing at most one letter.

There are four cases for this -
- *delete*: Delete a letter
```
'pick' => ['ick', 'pck', 'pik', 'pic']
```
- *transpose*: Exchange any two letters
```
'pick' => ['ipck', 'pcik', 'pikc']
```
- *replace*: replace a letter with any other letter
```
'pick' => ['aick', 'bick', 'cick', ..., 'piyk', 'pizk']
```
- *insert*: Insert a letter at any position
```
'pick' => ['apick', 'bpick', 'cpick', ..., 'picky', 'pickz']
```

In [69]:
word = 'tht'

splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]

print ('splits', splits)

splits [('', 'tht'), ('t', 'ht'), ('th', 't'), ('tht', '')]


In [71]:
deletes = [a + b[1:] for a,b in splits if b]

print ('deletes', deletes)

deletes ['ht', 'tt', 'th']


In [70]:
transposes = [a + b[1] + b[0] + b[2:] for a,b in splits if len(b) > 1 ]

print ('transposes', transposes)

transposes ['htt', 'tth']


In [72]:
import string


replaces = [a + c  + b[1:] for a,b in splits for c in string.ascii_lowercase if b ]
print ('replaces', replaces)

replaces ['aht', 'bht', 'cht', 'dht', 'eht', 'fht', 'ght', 'hht', 'iht', 'jht', 'kht', 'lht', 'mht', 'nht', 'oht', 'pht', 'qht', 'rht', 'sht', 'tht', 'uht', 'vht', 'wht', 'xht', 'yht', 'zht', 'tat', 'tbt', 'tct', 'tdt', 'tet', 'tft', 'tgt', 'tht', 'tit', 'tjt', 'tkt', 'tlt', 'tmt', 'tnt', 'tot', 'tpt', 'tqt', 'trt', 'tst', 'ttt', 'tut', 'tvt', 'twt', 'txt', 'tyt', 'tzt', 'tha', 'thb', 'thc', 'thd', 'the', 'thf', 'thg', 'thh', 'thi', 'thj', 'thk', 'thl', 'thm', 'thn', 'tho', 'thp', 'thq', 'thr', 'ths', 'tht', 'thu', 'thv', 'thw', 'thx', 'thy', 'thz']


In [53]:
import string


inserts = [a + c + b for a,b in splits for c in string.ascii_lowercase]
print (inserts)

['atht', 'btht', 'ctht', 'dtht', 'etht', 'ftht', 'gtht', 'htht', 'itht', 'jtht', 'ktht', 'ltht', 'mtht', 'ntht', 'otht', 'ptht', 'qtht', 'rtht', 'stht', 'ttht', 'utht', 'vtht', 'wtht', 'xtht', 'ytht', 'ztht', 'taht', 'tbht', 'tcht', 'tdht', 'teht', 'tfht', 'tght', 'thht', 'tiht', 'tjht', 'tkht', 'tlht', 'tmht', 'tnht', 'toht', 'tpht', 'tqht', 'trht', 'tsht', 'ttht', 'tuht', 'tvht', 'twht', 'txht', 'tyht', 'tzht', 'that', 'thbt', 'thct', 'thdt', 'thet', 'thft', 'thgt', 'thht', 'thit', 'thjt', 'thkt', 'thlt', 'thmt', 'thnt', 'thot', 'thpt', 'thqt', 'thrt', 'thst', 'thtt', 'thut', 'thvt', 'thwt', 'thxt', 'thyt', 'thzt', 'thta', 'thtb', 'thtc', 'thtd', 'thte', 'thtf', 'thtg', 'thth', 'thti', 'thtj', 'thtk', 'thtl', 'thtm', 'thtn', 'thto', 'thtp', 'thtq', 'thtr', 'thts', 'thtt', 'thtu', 'thtv', 'thtw', 'thtx', 'thty', 'thtz']


## Putting it togther

We can combine the four types of corrected words to get a list of all possible corrections.

In [73]:
def edits1(word):
    splits = [(word[:i], word[i:]) for i in range(len(word) + 1)]
    
    deletes = [a + b[1:] for a,b in splits if b]
    transposes = [a + b[1] + b[0] + b[2:] for a,b in splits if len(b) > 1 ]
    replaces = [a + c  + b[1:] for a,b in splits for c in string.ascii_lowercase if b ]
    inserts = [a + c + b for a,b in splits for c in string.ascii_lowercase]
    
    return set(deletes + transposes + replaces + inserts)

print ([word for word in edits1('spll') if word in word_counts] ) 

['sell', 'sill', 'spell', 'spill', 'soll']


Using the `word_counts` calculated earlier, we can suggest the most frequently occuring correct word as what the user might have wanted to type.

In [65]:
def get_word_count_or_one(word):
    return word_counts.get(word, 1)

def correct(word):
    candidates = [e for e in edits1(word) if e in word_counts]
    return max(candidates, key=get_word_count_or_one)

correct('spellin')

'spelling'